## Web Scraping from IMDB Site

In [1]:
import pandas as pd
import numpy as np

import requests
from requests import get
from bs4 import BeautifulSoup 

from time import sleep
from random import randint

In [6]:
# creating lists
titles = []
years = []
time = []
imdb_ratings = []
metascores = []
votes = []
us_gross = []

In [3]:
# Getting English translated titles from the movies
headers = {'Accept-Language': 'en-US, en;q=0.5'}

There are 1000 movies and each page has 50 movies listed.

So the first 50 movies' url: https://www.imdb.com/search/title/?groups=top_1000&ref_=adv_prv

Movies from 51 to 100: https://www.imdb.com/search/title/?groups=top_1000&start=51&ref_=adv_nxt

Movies from 101 to 150: https://www.imdb.com/search/title/?groups=top_1000&start=101&ref_=adv_nxt

In [4]:
pages = np.arange(1, 1001, 50)
pages

array([  1,  51, 101, 151, 201, 251, 301, 351, 401, 451, 501, 551, 601,
       651, 701, 751, 801, 851, 901, 951])

In [8]:
# Storing each of the urls of 50 movies 
for page in pages:
    # Getting the contents from the each url
    page = requests.get('https://www.imdb.com/search/title/?groups=top_1000&start=' + str(page) + '&ref_=adv_nxt', headers=headers)
    soup = BeautifulSoup(page.text, 'html.parser')
    
    # Aiming the part of the html we want to get the information from
    movie_div = soup.find_all('div', class_='lister-item mode-advanced')
    
    # Controling the loop’s rate by pausing the execution of the loop for a specified amount of time
    # Waiting time between requests for a number between 2-10 seconds
    sleep(randint(2,10))
    
    for container in movie_div:
        # Scraping the movie's name
        name = container.h3.a.text
        titles.append(name)
        
        # Scraping the movie's year
        year = container.h3.find('span', class_='lister-item-year').text
        years.append(year)
        
        # Scraping the movie's length
        runtime = container.find('span', class_='runtime').text if container.p.find('span', class_='runtime') else '-'
        time.append(runtime)
        
        # Scraping the rating
        imdb = float(container.strong.text)
        imdb_ratings.append(imdb)
        
        # Scraping the metascore
        m_score = container.find('span', class_='metascore').text if container.find('span', class_='metascore') else '-'
        metascores.append(m_score)
        
        # Scraping votes and gross earnings
        nv = container.find_all('span', attrs={'name':'nv'})
        vote = nv[0].text
        votes.append(vote)
        grosses = nv[1].text if len(nv) > 1 else '-'
        us_gross.append(grosses)

### Creating the Data Set

In [9]:
movies = pd.DataFrame({'movie':titles,
                       'year':years,
                       'time_minute':time,
                       'imdb_rating':imdb_ratings,
                       'metascore':metascores,
                       'vote':votes,
                       'gross_earning':us_gross})

In [10]:
movies.head()

,movie,year,time_minute,imdb_rating,metascore,vote,gross_earning
0,Dune,(2021),155 min,8.5,75,"36,140",-
1,Shang-Chi and the Legend of the Ten Rings,(2021),132 min,7.9,71,"86,041",-
2,The Matrix,(1999),136 min,8.7,73,"1,757,511",$171.48M
3,Shershaah,(2021),135 min,8.8,-,"101,538",-
4,Avengers: Endgame,(2019),181 min,8.4,78,"938,601",$858.37M


In [12]:
movies.dtypes

movie             object
year              object
time_minute       object
imdb_rating      float64
metascore         object
vote              object
gross_earning     object
dtype: object

In [13]:
# Cleaning 'year' column
movies['year'] = movies['year'].str.extract('(\d+)').astype(int)
movies.head(3)

,movie,year,time_minute,imdb_rating,metascore,vote,gross_earning
0,Dune,2021,155 min,8.5,75,"36,140",-
1,Shang-Chi and the Legend of the Ten Rings,2021,132 min,7.9,71,"86,041",-
2,The Matrix,1999,136 min,8.7,73,"1,757,511",$171.48M


In [15]:
# Cleaning 'time_minute' column
movies['time_minute'] = movies['time_minute'].str.extract('(\d+)').astype(int)
movies.head(3)

,movie,year,time_minute,imdb_rating,metascore,vote,gross_earning
0,Dune,2021,155,8.5,75,"36,140",-
1,Shang-Chi and the Legend of the Ten Rings,2021,132,7.9,71,"86,041",-
2,The Matrix,1999,136,8.7,73,"1,757,511",$171.48M


In [16]:
# Cleaning 'metascore' column
movies['metascore'] = movies['metascore'].str.extract('(\d+)')
# convert it to float and if there are dashes turn it into NaN
movies['metascore'] = pd.to_numeric(movies['metascore'], errors='coerce')

In [17]:
# Cleaning 'vote' column
movies['vote'] = movies['vote'].str.replace(',', '').astype(int)
movies.head(3)

,movie,year,time_minute,imdb_rating,metascore,vote,gross_earning
0,Dune,2021,155,8.5,75.0,36140,-
1,Shang-Chi and the Legend of the Ten Rings,2021,132,7.9,71.0,86041,-
2,The Matrix,1999,136,8.7,73.0,1757511,$171.48M


In [18]:
# Cleaning 'gross_earning' column
# left strip $ and right strip M 
movies['gross_earning'] = movies['gross_earning'].map(lambda x: x.lstrip('$').rstrip('M'))
# convert it to float and if there are dashes turn it into NaN
movies['gross_earning'] = pd.to_numeric(movies['gross_earning'], errors='coerce')
movies.head(3)

,movie,year,time_minute,imdb_rating,metascore,vote,gross_earning
0,Dune,2021,155,8.5,75.0,36140,NaN
1,Shang-Chi and the Legend of the Ten Rings,2021,132,7.9,71.0,86041,NaN
2,The Matrix,1999,136,8.7,73.0,1757511,171.48


### FINAL DATASET

In [20]:
movies

,movie,year,time_minute,imdb_rating,metascore,vote,gross_earning
0,Dune,2021,155,8.5,75.0,36140,NaN
1,Shang-Chi and the Legend of the Ten Rings,2021,132,7.9,71.0,86041,NaN
2,The Matrix,1999,136,8.7,73.0,1757511,171.48
3,Shershaah,2021,135,8.8,NaN,101538,NaN
4,Avengers: Endgame,2019,181,8.4,78.0,938601,858.37
...,...,...,...,...,...,...,...
1995,Kai Po Che,2013,120,7.8,40.0,35327,1.12
1996,Vizontele,2001,110,8.0,NaN,35210,NaN
1997,The Breath,2009,128,8.0,NaN,32788,NaN
1998,Andaz Apna Apna,1994,160,8.1,NaN,51710,NaN


### SAVING THE DATA INTO CSV FORMAT

In [21]:
movies.to_csv('movies.csv')

In [ ]:
# movies.csv is successfully saved in the current folder.